In [1]:
import numpy as np # linear algebra
import pandas as pd

In [2]:
train=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Training Dataset.csv')
train

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [3]:
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [4]:
x=train.iloc[:,:-1]
x.drop(columns=['Loan_ID'],inplace=True)
y=train.loc[:,'Loan_Status']
y=y.map({'Y':1,'N':0})

In [5]:
obj_col = x.select_dtypes(include=['object']).columns
int_col = x.select_dtypes(include=['int64','float64']).columns


In [6]:
from sklearn.impute import SimpleImputer

imp1 = SimpleImputer(strategy='mean')
imp1.fit(x[int_col])
x[int_col] = imp1.transform(x[int_col])

imp2 = SimpleImputer(strategy='most_frequent')
imp2.fit(x[obj_col])
x[obj_col] = imp2.transform(x[obj_col])

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in obj_col:
    x[col] = le.fit_transform(x[col])

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x)
x=scaler.transform(x)

In [10]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
#from catboost import CatBoostClassifier

In [11]:
xgb_clf = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    subsample=0.8,
    n_estimators=100,
    max_depth=8,
    learning_rate=0.01,
    colsample_bytree=1.0
)
xgb_clf.fit(x,y)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [10:11:16] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=1.0, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [18]:
test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Test Dataset.csv')
temp=test[['Loan_ID']]
test.drop(columns=['Loan_ID'],inplace=True)

imp1 = SimpleImputer(strategy='mean')
imp1.fit(test[int_col])
test[int_col] = imp1.transform(test[int_col])

imp2 = SimpleImputer(strategy='most_frequent')
imp2.fit(test[obj_col])
test[obj_col] = imp2.transform(test[obj_col])

for col in obj_col:
    test[col] = le.fit_transform(test[col])

test=scaler.fit_transform(test)

y_pred=xgb_clf.predict(test)

temp['Loan_Status']=y_pred
temp['Loan_Status'] = temp['Loan_Status'].map({1: 'Y', 0: 'N'})

temp.to_csv('/content/drive/MyDrive/Colab Notebooks/Sample_Submission (1).csv')